# Advanced 3D Reconstruction Visualization for Interior Reimaginer

This notebook demonstrates advanced 3D visualization techniques for the Interior Reimaginer project, highlighting superior alternatives to standard Plotly visualizations. We'll showcase GAN and diffusion-based depth enhancement methods for improved 3D reconstruction quality.

## Setup

First, let's clone the repository and install the required dependencies:

In [ ]:
# Clone the repository
!git clone https://github.com/your-username/CV_Project.git

%cd CV_Project

In [ ]:
# Install dependencies
!pip install -r interior_reimaginer/requirements.txt

# Additional packages for advanced visualization
!pip install open3d matplotlib plotly pyrender trimesh pyglet==1.5.27 PyOpenGL PyOpenGL-accelerate ipywidgets ipyvolume k3d pythreejs

## Import Required Modules

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import torch
import open3d as o3d
from PIL import Image
import cv2
import time
import io
import base64
from IPython.display import HTML, display
import ipyvolume as ipv
import plotly.graph_objects as go
from IPython.display import IFrame

# Import our project modules
from interior_reimaginer.models.interior_reimaginer import InteriorReimaginer
from interior_reimaginer.models.reconstruction_3d import DepthReconstructor

# Check if CUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
# Initialize models
print("Initializing models...")
reimaginer = InteriorReimaginer(device=device)
depth_reconstructor = DepthReconstructor()
print("Models initialized successfully!")

## Upload or Load an Image

You can either upload your own interior image or use one of our sample images:

In [ ]:
# Option 1: Upload an image
from google.colab import files
uploaded = files.upload()  # This will prompt you to upload an image

import io
image_name = list(uploaded.keys())[0]
image = Image.open(io.BytesIO(uploaded[image_name]))

# Display the uploaded image
plt.figure(figsize=(12, 8))
plt.imshow(image)
plt.title("Uploaded Image")
plt.axis('off')
plt.show()

## Process the Image

We'll process the image to extract depth information and other features:

In [ ]:
print("Processing the image...")
processed = reimaginer.image_processor.process_image(image)
print("Image processing complete!")

# Display the depth map
if processed.depth_map is not None:
    plt.figure(figsize=(12, 8))
    plt.imshow(processed.depth_map, cmap='inferno')
    plt.title("Generated Depth Map")
    plt.colorbar(label="Depth")
    plt.axis('off')
    plt.show()
else:
    print("Depth map generation failed.")

## Advanced 3D Visualization Methods

Let's explore different advanced visualization techniques beyond standard Plotly:

### 1. Enhanced Depth Map Visualization

In [ ]:
# Apply high-quality render with improved colormaps
enhanced_depth_viz = depth_reconstructor.render_depth_map(
    processed.depth_map, 
    colormap=cv2.COLORMAP_TURBO,  # Try different colormaps: COLORMAP_TURBO, COLORMAP_JET, COLORMAP_VIRIDIS
    width=1000, 
    height=800
)

plt.figure(figsize=(16, 10))
plt.imshow(enhanced_depth_viz)
plt.title("Enhanced Depth Map Visualization")
plt.axis('off')
plt.show()

### 2. GAN-Based Enhanced 3D Reconstruction

Use GAN-based depth enhancement to improve the quality of the 3D reconstruction:

In [ ]:
print("Creating GAN-enhanced 3D reconstruction...")
gan_render_img, gan_pcd = depth_reconstructor.enhanced_reconstruction(
    depth_map=processed.depth_map,
    image=image,
    width=1000,
    height=800,
    use_gan=True,  # Enable GAN enhancement
    solid_rendering=True  # Use solid rendering for more realistic visualization
)

plt.figure(figsize=(16, 10))
plt.imshow(gan_render_img)
plt.title("GAN-Enhanced 3D Reconstruction")
plt.axis('off')
plt.show()

### 3. Diffusion-Based Enhanced 3D Reconstruction

Use diffusion-based depth enhancement for a different approach to improving 3D reconstruction:

In [ ]:
print("Creating diffusion-enhanced 3D reconstruction...")
diffusion_render_img, diffusion_pcd = depth_reconstructor.diffusion_reconstruction(
    depth_map=processed.depth_map,
    image=image,
    width=1000,
    height=800,
    solid_rendering=True  # Use solid rendering for more realistic visualization
)

plt.figure(figsize=(16, 10))
plt.imshow(diffusion_render_img)
plt.title("Diffusion-Enhanced 3D Reconstruction")
plt.axis('off')
plt.show()

### 4. Local Region Models (LRM) Reconstruction

Use LRM for more detailed local geometry processing:

In [ ]:
print("Creating LRM 3D reconstruction...")
lrm_render_img, lrm_pcd = depth_reconstructor.lrm_reconstruction(
    depth_map=processed.depth_map,
    image=image,
    width=1000,
    height=800,
    downsample_factor=2,
    patch_size=32,
    overlap=8,
    solid_rendering=True
)

plt.figure(figsize=(16, 10))
plt.imshow(lrm_render_img)
plt.title("LRM-Enhanced 3D Reconstruction")
plt.axis('off')
plt.show()

## Interactive 3D Visualization with HTML Export

In [ ]:
def create_interactive_pointcloud_visualization(pcd, width=900, height=600):
    """Create an interactive 3D visualization using Plotly"""
    # Convert Open3D point cloud to numpy arrays
    points = np.asarray(pcd.points)
    colors = np.asarray(pcd.colors)
    
    # Convert RGB to hex for Plotly
    hex_colors = [f'rgb({int(r*255)},{int(g*255)},{int(b*255)})' for r, g, b in colors]
    
    # Create a 3D scatter plot
    fig = go.Figure(data=[go.Scatter3d(
        x=points[:, 0],
        y=points[:, 2],  # Use z as y for better orientation
        z=-points[:, 1],  # Negative y for correct orientation
        mode='markers',
        marker=dict(
            size=2,
            color=hex_colors,
            opacity=0.8
        )
    )])
    
    # Set up the layout
    fig.update_layout(
        width=width,
        height=height,
        scene=dict(aspectmode='data'),
        margin=dict(l=0, r=0, b=0, t=30),
        title="Interactive 3D Reconstruction"
    )
    
    return fig

# Create interactive visualization using the best point cloud
best_pcd = lrm_pcd if len(lrm_pcd.points) > 0 else (diffusion_pcd if len(diffusion_pcd.points) > 0 else gan_pcd)

if len(best_pcd.points) > 0:
    interactive_fig = create_interactive_pointcloud_visualization(best_pcd)
    interactive_fig.show()
else:
    print("No valid point cloud available for interactive visualization")

## Conclusion

In this notebook, we've demonstrated several advanced 3D visualization techniques for interior reconstruction:

1. Enhanced depth map rendering with improved colormaps
2. GAN-based depth enhancement for better 3D reconstruction
3. Diffusion-based methods for filling in missing depth information
4. Local Region Models (LRM) for detailed geometry processing
5. Interactive 3D visualizations using various libraries

These techniques provide significant improvements over standard Plotly visualizations, offering better detail, more realistic rendering, and enhanced interactivity for exploring 3D reconstructions.